<a href="https://colab.research.google.com/github/Decode333/face-recognition-siamese/blob/main/face_recognition_siamese.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import numpy as np
import tensorflow as tf
import pickle
import os
from tensorflow.keras import layers, Model
from tensorflow.keras.preprocessing.image import img_to_array, load_img
import random

In [3]:
def load_images_from_folder(folder_path, image_size=(100, 100)):
    images = []
    labels = []
    for person_name in os.listdir(folder_path):
        person_folder = os.path.join(folder_path, person_name)
        if not os.path.isdir(person_folder):
            continue
        for img_name in os.listdir(person_folder):
            img_path = os.path.join(person_folder, img_name)
            img = load_img(img_path, target_size=image_size)
            img = img_to_array(img) / 255.0
            images.append(img)
            labels.append(person_name)
    return np.array(images), np.array(labels)

In [4]:
def create_pairs(images, labels):
    pair_images = []
    pair_labels = []

    # Get unique class labels
    unique_labels = np.unique(labels)

    label_to_indices = {label: np.where(labels == label)[0] for label in unique_labels}

    for idx, img in enumerate(images):
        current_label = labels[idx]
        pos_idx = idx
        while pos_idx == idx:
            pos_idx = random.choice(label_to_indices[current_label])

        neg_label = random.choice(unique_labels[unique_labels != current_label])
        neg_idx = random.choice(label_to_indices[neg_label])

        # Positive pair
        pair_images.append([img, images[pos_idx]])
        pair_labels.append(1)

        # Negative pair
        pair_images.append([img, images[neg_idx]])
        pair_labels.append(0)

    return np.array(pair_images), np.array(pair_labels)

In [ ]:
# --- Load and prepare dataset ---
data_folder = "dataset"  # Change if your dataset folder is different
images, labels = load_images_from_folder(data_folder)
pair_images, pair_labels = create_pairs(images, labels)

In [ ]:
# Split into train and validation sets
split_idx = int(0.8 * len(pair_images))
trainX, testX = pair_images[:split_idx], pair_images[split_idx:]
trainY, testY = pair_labels[:split_idx], pair_labels[split_idx:]

In [9]:
def build_siamese_network(input_shape=(100, 100, 3)):
    embedding_model = build_embedding_model(input_shape)

    input_a = layers.Input(shape=input_shape)
    input_b = layers.Input(shape=input_shape)

    emb_a = embedding_model(input_a)
    emb_b = embedding_model(input_b)

    distance = layers.Lambda(lambda tensors: tf.math.abs(tensors[0] - tensors[1]))([emb_a, emb_b])
    outputs = layers.Dense(1, activation="sigmoid")(distance)

    siamese_model = Model([input_a, input_b], outputs)

    return siamese_model

def contrastive_loss(y_true, y_pred, margin=1.0):
    """
    Contrastive loss function.
    """
    y_true = tf.cast(y_true, tf.float32)
    square_pred = tf.math.square(y_pred)
    margin_square = tf.math.square(tf.math.maximum(margin - y_pred, 0))
    return tf.math.reduce_mean(y_true * square_pred + (1 - y_true) * margin_square)

In [ ]:
# --- Build Siamese model ---
input_shape = images.shape[1:]
model = build_siamese_network(input_shape)
model.compile(loss=contrastive_loss, optimizer="adam", metrics=["accuracy"])

In [ ]:
# --- Train the model ---
history = model.fit(
    [trainX[:, 0], trainX[:, 1]],
    trainY,
    validation_data=([testX[:, 0], testX[:, 1]], testY),
    batch_size=32,
    epochs=20
)

In [ ]:
# --- Save the entire model ---
model.save("siamese_model.h5")

In [ ]:
# --- Save the embeddings for known faces ---
embedding_model = model.get_layer('Embedding')
known_embeddings = []
known_names = []

# Create known database
for label in np.unique(labels):
    person_indices = np.where(labels == label)[0]
    person_img = images[person_indices[0]]  # Take one image per person
    person_img = np.expand_dims(person_img, axis=0)
    emb = embedding_model.predict(person_img)[0]
    known_embeddings.append(emb)
    known_names.append(label)

# Save to pickle
with open("embeddings.pickle", "wb") as f:
    pickle.dump({"embeddings": known_embeddings, "names": known_names}, f)